In [167]:
# %pip install scikit-learn

In [168]:
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import pickle as pk
import pandas as pd
from tensorflow import keras
import scipy
np.random.seed(0)

In [169]:
with open("../data/pickles/data_slidingwindowFalse10_samplingrate20_final_df.p", 'rb') as f:
  data = pk.load(f)
dataframe = data['data']
# test_dataframe = dataframe[dataframe['pid'] == "BK7610"]
test_dataframe = dataframe[dataframe['pid'] == 'BU4707']
dataframe = dataframe[(dataframe['pid'] != 'BK7610') & (dataframe['pid']!='BU4707')]


dataframe['pid'].unique()

array(['DC6359'], dtype=object)

In [170]:
# test_dataframe['tac'].unique()

In [171]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(scipy.stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)



In [172]:

# scale_columns = ['x', 'y', 'z']
# scaler = RobustScaler()
# scaler = scaler.fit(dataframe[scale_columns])
# dataframe.loc[:, scale_columns] = scaler.transform(
#   dataframe[scale_columns].to_numpy()
# )

In [189]:
TIME_STEPS = 20
STEP = 20

X_train, y_train = create_dataset(
    dataframe[['x', 'y', 'z']],
    dataframe.tac,
    TIME_STEPS,
    STEP
)


X_test, y_test = create_dataset(
    test_dataframe[['x', 'y', 'z']],
    test_dataframe.tac,
    TIME_STEPS,
    STEP
)

/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_21149/721208264.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(scipy.stats.mode(labels)[0][0])
/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_21149/721208264.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or

In [190]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((310559, 1, 3), (224319, 1, 3), (310559, 1), (224319, 1))

In [191]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [192]:
model = keras.Sequential()
# model.add(
#     keras.layers.Bidirectional(
#       keras.layers.LSTM(
#           units=256,
#           input_shape=[X_train.shape[1], X_train.shape[2]]
#       )
#     )
# )
model.add(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam', 
  metrics=['acc']
)


In [193]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01, 
    shuffle=False)   

Epoch 1/20
9608/9608 [==============================] - 28s 3ms/step - loss: 0.0357 - acc: 0.9919 - val_loss: 6.8162e-07 - val_acc: 1.0000
Epoch 2/20
9608/9608 [==============================] - 25s 3ms/step - loss: 0.0539 - acc: 0.9875 - val_loss: 1.1312e-05 - val_acc: 1.0000
Epoch 3/20
9608/9608 [==============================] - 26s 3ms/step - loss: 0.0539 - acc: 0.9881 - val_loss: 3.1064e-05 - val_acc: 1.0000
Epoch 4/20
9608/9608 [==============================] - 27s 3ms/step - loss: 0.0555 - acc: 0.9905 - val_loss: 3.8768e-05 - val_acc: 1.0000
Epoch 5/20
9608/9608 [==============================] - 26s 3ms/step - loss: 0.0501 - acc: 0.9920 - val_loss: 1.0025e-06 - val_acc: 1.0000
Epoch 6/20
9608/9608 [==============================] - 26s 3ms/step - loss: 0.0553 - acc: 0.9917 - val_loss: 5.6496e-08 - val_acc: 1.0000
Epoch 7/20
9608/9608 [==============================] - 28s 3ms/step - loss: 0.0531 - acc: 0.9926 - val_loss: 7.1072e-07 - val_acc: 1.0000
Epoch 8/20
9608/9608 [=====

In [194]:
# test_dataframe[test_dataframe['tac']==1].count(), test_dataframe[test_dataframe['tac']==0].count()

In [195]:
# X_test[0]

In [196]:
model.evaluate(X_test, y_test) 

7010/7010 [==============================] - 6s 817us/step - loss: 18.9527 - acc: 0.2846


[18.95272445678711, 0.28459471464157104]

In [ ]:
dataframe[dataframe['tac']==1].count(), dataframe[dataframe['tac']==0].count(), 

In [ ]:
dataframe.shape

In [ ]:
# test = np.squeeze(y_test)
np.where(np.squeeze(y_train))==1

In [ ]:
# len(np.where(test == 1))

In [ ]:
X_test[0]

In [ ]:
X_train[0]

In [ ]:
# y_train.shape

In [ ]:
# model.save("bidirectional_lstm.h5")

In [ ]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(y_train.shape[1], activation='sigmoid')))

model.compile(
  loss='binary_crossentropy',
  optimizer=keras.optimizers.Adam(0.01),
  metrics=['acc']
)


history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 


In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_test.shape

In [ ]:
X_test.shape

In [ ]:
test_dataframe.head()

## MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf = MLPClassifier(solver='adam', shuffle=True, random_state=1)

In [ ]:
X_train.shape

In [ ]:
clf.fit(np.reshape(X_train, (15542, 60)), y_train)
clf.get_params()

In [ ]:
X_test.shape

In [ ]:
print('Accuracy ', accuracy_score(y_test, clf.predict(np.reshape(X_test, (11229, 60)))))

## CNN 

In [ ]:
batch_size = 256
epochs = 20

flatten = keras.layers.Flatten()
conv_layer1 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
conv_layer2 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
dropout = keras.layers.Dropout(0.5)
max_pooling = keras.layers.MaxPool1D(pool_size=2)
fc_layer = keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = keras.layers.Dense(y_train.shape[1], activation = 'sigmoid')
base_model = keras.Sequential([
                                  conv_layer1,  
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

base_model.compile(loss='binary_crossentropy', 
                   optimizer='adam',
                   metrics=['acc']) 

In [ ]:
history = base_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 

In [ ]:
base_model.evaluate(X_test, y_test)